In [ ]:
import torch.nn as nn
import torch
from tqdm import trange
import matplotlib.pyplot as plt
import struct
import numpy as np
import torchvision

In [ ]:
training_data = torchvision.datasets.MNIST(root="mnist_data",train=True,download=True,transform=torchvision.transforms.ToTensor())
testing_data = torchvision.datasets.MNIST(root="mnist_data",train=False,download=True,transform=torchvision.transforms.ToTensor())

In [ ]:
x_train,y_train = training_data.data.reshape(-1,28*28).double(),training_data.targets
x_test,y_test = testing_data.data.reshape(-1,28*28).double(),testing_data.targets

In [ ]:
# Model
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(784,128)
        self.l2 = nn.Linear(128,10)
        self.act = nn.ReLU()
    
    def forward(self,x):
        x = self.act(self.l1(x))
        x = self.l2(x)
        return x

model = Net().double()

In [ ]:
model

In [ ]:
# Training
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
BS = 32
for _ in (t:= trange(1500,colour="green")):
     samp = torch.randint(0,x_train.shape[0],(BS,))
     optimizer.zero_grad()
     loss = loss_fn(model(x_train[samp]),y_train[samp])
     loss.backward()
     optimizer.step()
     out = model(x_train[samp]).softmax(dim=1)
     accuracy = (torch.argmax(out,dim=1) == y_train[samp]).double().mean()
     t.set_description(f"Loss = {loss:0.2f} Accuracy {accuracy.item():0.2f}")

In [ ]:
n = torch.randint(0, x_test.shape[0],(1,))
plt.title(f"pred {torch.argmax(model(x_test[n].reshape(1,-1)).softmax(dim=1),dim=1).item()}, label {y_test[n].item()}")
plt.imshow(x_test[n].reshape(28, 28),cmap="gray")